In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from bs4 import BeautifulSoup
import requests
import dateutil.parser

def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date

def money_to_int(moneystring):
    moneystring = moneystring.replace('$', '').replace(',', '')
    return int(moneystring)

In [2]:
all_data = pd.read_csv('Movies_and_CDC_Data.csv')

/Users/alexstake/opt/anaconda3/envs/metis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (2,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484115 entries, 0 to 484114
Data columns (total 80 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Date                                    484115 non-null  object 
 1   rank                                    484115 non-null  int64  
 2   rank_yest                               484115 non-null  object 
 3   movie_title                             484115 non-null  object 
 4   daily_gross                             484115 non-null  object 
 5   gross_change_day%                       484115 non-null  object 
 6   gross_change_week%                      484115 non-null  object 
 7   num_of_theaters                         484115 non-null  object 
 8   avg_gross_per_theater                   484115 non-null  object 
 9   gross_to_date                           484115 non-null  object 
 10  days_in_release                         4841